# VideoChat Inference Demonstration

This notebook demonstrates the core "behind-the-scenes" logic of the VideoChat application. We will walk through:
1.  **Setup**: Loading the necessary model and processor from Hugging Face.
2.  **Input Preparation**: Defining the video path and the question to ask.
3.  **Inference Function**: A look at the reusable function that runs a single pass of the model.
4.  **Multi-Perception Loop**: The iterative process where the model analyzes the video in steps, using context from previous steps to refine its focus.

**Hardware Requirement**: This notebook requires a CUDA-enabled NVIDIA GPU.

In [ ]:
import sys
import os
# Add the src directory to the system path to import modules
sys.path.append(os.path.abspath('../src'))

import torch
import re
import ast
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor
from my_vision_process import process_vision_info, client # Assuming client is None for local execution

# Check for GPU
if not torch.cuda.is_available():
    raise RuntimeError("Error: CUDA is not available. This notebook requires a GPU.")
print(f"CUDA is available. Using device: {torch.cuda.get_device_name(0)}")

## 1. Setup: Load Model and Processor

load the `OpenGVLab/VideoChat-R1_5` model and its associated processor. We use `bfloat16` for memory efficiency and `flash_attention_2` for faster performance if available.

In [ ]:
def setup_model():
    """Loads and returns the model and processor onto the GPU."""
    model_path = "OpenGVLab/VideoChat-R1_5"
    print(f"Loading model from {model_path}...")
    
    try:
        import flash_attn
        attn_implementation = "flash_attention_2"
        print("flash-attn is available, using 'flash_attention_2'.")
    except ImportError:
        print("flash-attn not installed. Falling back to 'sdpa'.")
        attn_implementation = "sdpa"

    model = Qwen3VLForConditionalGeneration.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        device_map="cuda",
        attn_implementation=attn_implementation
    ).eval()
    processor = AutoProcessor.from_pretrained(model_path)
    print("Model and processor loaded successfully.")
    return model, processor

model, processor = setup_model()

## 2. Input Preparation

Define the path to your video and the question you want to ask. We also define the prompt templates used by the application. The `GLUE` prompt asks the model to identify relevant time clips, which are used in subsequent iterations.

In [ ]:
# --- IMPORTANT --- 
# Change this to the path of a video file on your system.
# For example: "videos/my_cool_video.mp4"
VIDEO_PATH = "path/to/your/video.mp4"
QUESTION = "What is the person in the video doing?"
NUM_PERCEPTIONS = 3

if not os.path.exists(VIDEO_PATH):
    print(f"ERROR: Video file not found at '{VIDEO_PATH}'. Please update the VIDEO_PATH variable.")

QA_THINK_GLUE = """Answer the question: "[QUESTION]" according to the content of the video. \n
Output your think process within the <think> </think> tags.\n\nThen, provide your answer within the <answer> </answer> tags. At the same time, in the <glue> </glue> tags, present the precise time period in seconds of the video clips on which you base your answer in the format of [(s1, e1), (s2, e2), ...]. For example: <think>...</think><answer>A</answer><glue>[(5.2, 10.4)]</glue>."""

QA_THINK = """Answer the question: "[QUESTION]" according to the content of the video.\n\nOutput your think process within the <think> </think> tags.\n\nThen, provide your answer within the <answer> </answer> tags. For example: <think>...</think><answer>A</answer>."""

## 3. The Core Inference Function

This function encapsulates a single interaction with the model. It takes the video, a prompt, and optional `pred_glue` (time clips from a previous step), processes them, and returns the model's raw text output.

In [ ]:
def inference(video_path, prompt, model, processor, pred_glue=None, max_new_tokens=2048):
    """Runs a single inference pass on the model."""
    messages = [
        {"role": "user", "content": [
                {"type": "video", 
                "video": video_path,
                'key_time': pred_glue,
                "total_pixels": 128*12 * 28 * 28, 
                "min_pixels": 128 * 28 * 28,
                },
                {"type": "text", "text": prompt},
            ]
        },
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs, video_kwargs = process_vision_info(messages, return_video_kwargs=True, client=client)
    fps_inputs = video_kwargs['fps'][0]
    inputs = processor(text=[text], images=image_inputs, videos=video_inputs, fps=fps_inputs, padding=True, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)

    generated_ids = [output_ids[i][len(inputs.input_ids[i]):] for i in range(len(output_ids))]
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text[0]

## 4. The Multi-Perception Loop

This is the main logic. We loop for `NUM_PERCEPTIONS` iterations. In each step (except the last), we use the `GLUE` prompt to ask for relevant time clips. We extract these clips and feed them back into the next iteration via the `pred_glue` argument, allowing the model to focus its attention.

In [ ]:
answers = []
pred_glue = None

print(f"--- Starting Inference for '{VIDEO_PATH}' ---")

for perception in range(NUM_PERCEPTIONS):
    print(f"\n>>> Perception Iteration {perception + 1}/{NUM_PERCEPTIONS} <<<")

    # Use the GLUE prompt for all but the last iteration
    if perception < NUM_PERCEPTIONS - 1:
        current_prompt = QA_THINK_GLUE.replace("[QUESTION]", QUESTION)
    else:
        current_prompt = QA_THINK.replace("[QUESTION]", QUESTION)
    
    print(f"[Prompt]:\n{current_prompt}\n")
    if pred_glue:
        print(f"[Input Glue]: Focusing on time clips {pred_glue}\n")

    # Run inference
    ans = inference(
        VIDEO_PATH, current_prompt, model, processor,
        pred_glue=pred_glue
    )

    print(f"[Model Raw Output]:\n{ans}\n")
    answers.append(ans)
    
    # Reset glue and try to parse it from the latest answer
    pred_glue = None
    try:
        pattern_glue = r'<glue>(.*?)</glue>'
        match_glue = re.search(pattern_glue, ans, re.DOTALL)
        if match_glue:
            glue_str = match_glue.group(1).strip()
            pred_glue = ast.literal_eval(glue_str)
            print(f"[Output Glue Found]: Extracted {pred_glue} for next iteration.")
        else:
            print("[Output Glue Found]: None.")
    except Exception as e:
        print(f"Could not parse glue from output: {e}")

## 5. Final Answer

The final answer is the output from the last iteration of the loop.

In [ ]:
final_answer = answers[-1] if answers else "No answer was generated."
print("\n--- FINAL ANSWER ---")
print(final_answer)